In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel('Data_Train.xlsx')

The dataset is a delivery Prediction dataset where we are asked to predict the delivery time given specific attributes

In [ ]:
df.head(3)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes


In [ ]:
df.isna().sum()

Restaurant       0
Location         0
Cuisines         0
Average_Cost     0
Minimum_Order    0
Rating           0
Votes            0
Reviews          0
Delivery_Time    0
dtype: int64

In [ ]:
print("The total number of values is",len(df))
df.nunique(axis = 0)

The total number of values is 11094


Restaurant       7480
Location           35
Cuisines         2179
Average_Cost       26
Minimum_Order      18
Rating             33
Votes            1103
Reviews           761
Delivery_Time       7
dtype: int64

All these features look like they dont have null values in them but on observing each of them individually we could see they have null values

In [ ]:
df['Rating'].value_counts()

-                     1191
3.7                    869
3.6                    846
3.5                    818
3.8                    800
NEW                    758
3.9                    749
3.4                    718
3.3                    675
4.0                    614
3.2                    511
4.1                    459
3.1                    411
3.0                    302
4.2                    272
4.3                    247
2.9                    199
2.8                    157
4.4                    142
4.5                     78
2.7                     76
2.6                     42
4.6                     41
4.7                     36
2.5                     27
2.4                     13
4.8                     13
Opening Soon            12
4.9                      8
2.3                      6
Temporarily Closed       2
2.2                      1
2.1                      1
Name: Rating, dtype: int64

The operning soon and temporarily closed are not null values but they're not rating values so we need to replace them

In [ ]:
df["Average_Cost"].value_counts()

₹200      3241
₹100      2557
₹150      2462
₹250       881
₹300       537
₹350       283
₹400       282
₹50        265
₹600       154
₹500       101
₹450        63
₹550        60
₹650        55
₹800        44
₹750        38
₹900        15
₹700        15
₹1,000      12
₹850        12
₹1,200       8
₹950         4
₹1,150       1
for          1
₹1,100       1
₹1,400       1
₹2,050       1
Name: Average_Cost, dtype: int64

There is a value called for in Average_Cost we need to replace them. We also have to remove '₹' and ' , ' in these values so we can convert them into integer values

In [ ]:
df["Minimum_Order"].value_counts()

₹50     10118
₹99       779
₹0        158
₹199        8
₹200        8
₹299        3
₹59         3
₹350        3
₹79         2
₹90         2
₹450        2
₹300        2
₹400        1
₹250        1
₹150        1
₹240        1
₹89         1
₹500        1
Name: Minimum_Order, dtype: int64

We need to remove '₹' from the Minimu_Order all these values

In [ ]:
df["Votes"].value_counts()

-       2074
4        248
6        200
7        182
9        181
        ... 
1106       1
935        1
663        1
1116       1
3700       1
Name: Votes, Length: 1103, dtype: int64

There are no null values in votes column. But there is a value '-' in this so we need to remove before trying to fill the values or before making the type to int/flot

In [ ]:
df["Reviews"].value_counts()

-       2312
2        420
3        387
1        381
4        356
        ... 
1501       1
983        1
343        1
397        1
549        1
Name: Reviews, Length: 761, dtype: int64

There are no null values in Review column. But there is a value '-' in this so we need to remove before trying to fill the values or before making the type to int/flot

In [ ]:
df['Delivery_Time']

0        30 minutes
1        30 minutes
2        65 minutes
3        30 minutes
4        65 minutes
            ...    
11089    30 minutes
11090    30 minutes
11091    30 minutes
11092    30 minutes
11093    30 minutes
Name: Delivery_Time, Length: 11094, dtype: object

We need to remove minutes from Delivery_Time feature and also convert it into int/float

In [ ]:
df['Delivery_Time'] = df['Delivery_Time'].map(lambda x: x[0:2]).astype(int)
df['Delivery_Time']

0        30
1        30
2        65
3        30
4        65
         ..
11089    30
11090    30
11091    30
11092    30
11093    30
Name: Delivery_Time, Length: 11094, dtype: int64

In [ ]:
df['Cuisines'].nunique()

2179

There are 2179 unique values in it but since each value has multiple food we can't treat this as 1 categorical feature

In [ ]:
a=df["Cuisines"]
food={}
k=1

for i in df['Cuisines']:
    for j in i.split(","):
      j=''.join(j.split())

      if food.get(j,0) == 0:
          food[j]=k
          k+=1
len(food)

101

There are 101 types of unique food in cuisines features. So we can take each food value and change it into a column instead of treating each value in the column as 1 cuisine

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def preprocess(daf):

    # chganging the type of rating feature
    for i in range(len(daf)):
            try:
                daf["Rating"][i]=float(daf["Rating"][i])
            except:
                daf["Rating"][i]=None
            
    bad_chars=[",","₹"]

    #removing "," and "₹"  from "Average_Cost","Minimum_Order","Votes" and "Reviews" and making it float
    def func(vals):
        for val in vals:
            for i in range(len(daf)):
                for j in bad_chars:
                    daf[val][i]=daf[val][i].replace(j,"")
                try:
                    daf[val][i]=float(daf[val][i])
                except:
                    daf[val][i]=None
    vals=["Average_Cost","Minimum_Order","Votes","Reviews"]
    func(vals)



    #using the food dictionary to turn each food item in the "Cusines" column into a new feature 
    table=[[0 for i in range(101)] for k in range(len(daf))]
    table2=[[0 for i in range(35)] for k in range(len(daf))]    
    for i in range(len(daf)):
        for j in daf["Cuisines"][i].split(","):
            while " " in j:
                j=j.replace(" ","")
            table[i][food[j]-1]=1    
    table=pd.DataFrame(table,columns=food)
    daf=daf.join(table)



    #Since we've added the food items to the dataset we can remove the "Cuisinses" feature
    #Out of 11094 Restaurant has 7480 unique values and since all of them are id's we cant use them
    daf.drop(["Cuisines"],axis=1,inplace=True)
    daf.drop(["Restaurant"],axis=1,inplace=True)


    #Now that most of the features are float values we can use the mean to fill them
    daf=daf.fillna(daf.mean())

    return daf

In [ ]:
df=preprocess(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launche

In [ ]:
df.head()

,Location,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,FastFood,Rolls,Burger,...,Nepalese,Indian,Tex-Mex,Hotdogs,SouthAmerican,MiddleEastern,African,Turkish,Israeli,Greek
0,"FTI College, Law College Road, Pune",200.0,50.0,3.5,12.0,4.0,30,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,"Sector 3, Marathalli",100.0,50.0,3.5,11.0,4.0,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Mumbai Central,150.0,50.0,3.6,99.0,30.0,65,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Sector 1, Noida",250.0,99.0,3.7,176.0,95.0,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rmz Centennial, I Gate, Whitefield",200.0,99.0,3.2,521.0,235.0,65,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The last thing we need to either convert or drop the Location attribute in the dataset

In [18]:
df['Location'].nunique()

35

In [19]:
a=df["Location"].values
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(a)

onehot_encoder = OneHotEncoder(sparse=False)

onehot_encoded = onehot_encoder.fit_transform(label_encoded.reshape(-1, 1))

onehot_encoded1=pd.DataFrame(onehot_encoded)
df=df.join(onehot_encoded1)
df.drop(columns = ['Location'],inplace = True)

df.head()

,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,FastFood,Rolls,Burger,Salad,...,25,26,27,28,29,30,31,32,33,34
0,200.0,50.0,3.5,12.0,4.0,30,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100.0,50.0,3.5,11.0,4.0,30,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,150.0,50.0,3.6,99.0,30.0,65,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,250.0,99.0,3.7,176.0,95.0,30,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,200.0,99.0,3.2,521.0,235.0,65,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
print("Length of total dataset",len(df))
df.drop_duplicates(inplace=True)
print("Length after dropping douplicate values",len(df))

Length of total dataset 11094
Length after dropping douplicate values 10666


We're creating new variable for target variable

In [21]:
y = df['Delivery_Time'].values
df.drop(columns = ['Delivery_Time'],inplace =True)

In [22]:
for i in df.columns:
  q1 = df[i].quantile(0.25)
  q3 = df[i].quantile(0.75)
  iqr = q3-q1
  print(i,sum((df[i] < q1-1.5*iqr) & (df[i] < q3+1.5*iqr)))

Average_Cost 260
Minimum_Order 158
Rating 166
Votes 0
Reviews 0
FastFood 0
Rolls 0
Burger 0
Salad 0
Wraps 0
IceCream 0
Desserts 0
Italian 0
StreetFood 0
Mughlai 0
NorthIndian 0
Chinese 0
Cafe 0
Beverages 0
SouthIndian 0
Thai 0
Asian 0
Mithai 0
Momos 0
Indonesian 0
Biryani 0
Kerala 0
Bihari 0
Mexican 0
Japanese 0
Bakery 0
Burmese 0
BubbleTea 0
Tea 0
Pizza 0
Lucknowi 0
Mangalorean 0
European 0
Continental 0
Sandwich 0
HealthyFood 0
Bengali 0
American 0
Mishti 0
Hyderabadi 0
Andhra 0
Assamese 0
Maharashtrian 0
German 0
Arabian 0
FingerFood 0
Kebab 0
Chettinad 0
Seafood 0
Juices 0
Parsi 0
Sushi 0
Odia 0
Tamil 0
Cantonese 0
NorthEastern 0
Tibetan 0
Lebanese 0
Spanish 0
BarFood 0
Konkan 0
Paan 0
Steak 0
Mediterranean 0
Bohri 0
Afghan 0
Goan 0
Gujarati 0
BBQ 0
RawMeats 0
Malaysian 0
Vietnamese 0
SriLankan 0
Rajasthani 0
Poké 0
RoastChicken 0
Coffee 0
Brazilian 0
Belgian 0
Naga 0
Korean 0
ModernIndian 0
Awadhi 0
Kashmiri 0
French 0
Portuguese 0
CharcoalChicken 0
FrozenYogurt 0
Bangladeshi 0
Ma

We could see that there are outliers in Minimum_Order,Average_cost and Rating columns. we would use winsorize method to remove outliers. The methods takes these output values and fixes them to a certain quantile limit

In [23]:
from scipy.stats.mstats import winsorize

df_win = winsorize(df,(0.02,0.02))

In [24]:
x = df_win.data    # x variables

In [25]:
y       #y or predictor varianle

array([30, 30, 65, ..., 30, 30, 30])

If you observe the target variable its clearly but the number of unique values in it is just 6. We can treat this as a classification problem or we can treat this as a regresson problem since these values are not continuous. 

This is Ordinal Regression. We can use a classification algorithm with a bit different implementation to use it for Ordinal Regression


(Reference :https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c)

In [33]:
from sklearn.base import clone


class OrdinalClassifier():
    
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
    
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
                
    def predict_proba(self, X):

        clfs_predict = {k:self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i,y in enumerate(self.unique_class):
          
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[i][:,1])
            elif i in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                predicted.append(clfs_predict[i-1][:,1] - clfs_predict[i][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[i-1][:,1])
        return np.vstack(predicted).T
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [51]:
from sklearn.tree import DecisionTreeClassifier

clf = OrdinalClassifier(DecisionTreeClassifier(max_depth = 9))

In [52]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,stratify = y)

In [53]:
clf.fit(x_train,y_train)

In [54]:
y_pred = clf.predict(x_test)

In [55]:
di = {0:10,1:12,2:20,3:30,4:45,5:65,6:80}

In [56]:
y_pred = [di[x] for x in y_pred]

In [57]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_pred,y_test)**(0.5)

12.07974906173817

Now let's test how a normal decisionTreeRegressor works against this

In [41]:
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor(max_depth = 9)
clf.fit(x_train,y_train)

DecisionTreeRegressor(max_depth=9)

In [42]:
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)

In [43]:
from sklearn.metrics import mean_squared_error

print("Train rmse",mean_squared_error(y_train,y_pred_train)**0.5)

print("Test rmse",mean_squared_error(y_test,y_pred_test)**0.5)

Train rmse 9.260977784802053
Test rmse 10.319948564287333


We could see that there is no improvement in rmse when ordinal regressor is used. So like in most machine learning problems its better to test and see how a model performs even though its meant for that specific problem